In [2]:
import re
import os
import numpy as np
from progress.bar import Bar
from tqdm import tqdm


In [3]:

def writeRadius(r1,r2):
    lpMesh = open("loop.geo", "r")
    loopMesh = lpMesh.read()
    lpMesh.close()

    reg1 = r"(%r1)"
    reg2 = r"(%r2)"

    sub1 = str(r1)
    sub2 = str(r2)

    loopMesh = re.sub(reg1,sub1,loopMesh,0,re.MULTILINE)
    loopMesh = re.sub(reg2,sub2,loopMesh,0,re.MULTILINE)

    try:
        lpTemp = open("loopTemp.geo", "x")
    except:
        os.remove("loopTemp.geo")
        lpTemp = open("loopTemp.geo","x")

    lpTemp.write(loopMesh)
    lpTemp.close()


    # write surface mesh
    lpMesh = open("loopFIELD.geo", "r")
    loopMesh = lpMesh.read()
    lpMesh.close()
    reg1 = r"(%r1)"
    reg2 = r"(%r2)"
    sub1 = str(r1)
    sub2 = str(r2)
    loopMesh = re.sub(reg1,sub1,loopMesh,0,re.MULTILINE)
    loopMesh = re.sub(reg2,sub2,loopMesh,0,re.MULTILINE)
    try:
        lpTemp = open("loopFIELDTemp.geo", "x")
    except:
        os.remove("loopFIELDTemp.geo")
        lpTemp = open("loopFIELDTemp.geo","x")
    lpTemp.write(loopMesh)
    lpTemp.close()

def runGMSH():
    print("Meshing geometry")
    os.system("gmsh loopFIELDTemp.geo -nopopup -0 -v 1")
    os.system("gmsh loopTemp.geo -nopopup -0 -v 1")
    try:
        os.system("rm *.geo_unrolled")
        os.remove("loopFIELDTemp.geo")
        os.remove("loopTemp.geo")
    except:
        print("nope")
    print("Done meshing\n")


def runTTH():
    print("Running tth settings_washer.json >/dev/null 2>&1")
    os.system("tth settings_washer.json >/dev/null 2>&1")
    print("Done\n")


def getCurrent():
    iFile = open("./output/i.txt")
    it = iFile.read()
    iFile.close()
    reg = r"fone fone .+ (\d*\.\d*e-\d*)"
    i_s = re.findall(reg,it,re.MULTILINE)[0]
    return i_s
    
def runNoiseEx(b_path,current):
    path = "../../cmake-build-debug/nex"
    print("Running "+path+" "+b_path+" "+current)
    val = os.popen(path+" "+b_path+" "+current).read()
    print("Done with tth!\n")
    return val[:-1]


def runSim():
    # First we change loop radius
    r2s = np.linspace(0.1e-6, 2e-6, 10)

# writeRadius(15e-6, 0.2e-6)

# runGMSH()

# runTTH()

# i = getCurrent()

# runNoiseEx("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk",i)


# runSim()

Meshing geometry
Done meshing

Running tth settings_washer.json >/dev/null 2>&1
Done

Running ../../cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk 1.0943227870e-01
Done with tth!



In [10]:

SPIN_DENS = 5e17
MU_B = 9.2740100783e-24
PHI_0 = 2.067833848e-15
MU_0 = 1.25663706212e-6 	
Analytic = lambda R, D : 2/3*MU_0**2*MU_B**2*SPIN_DENS*R/D*1/(PHI_0**2)

r1 = 15e-6
r2s = np.linspace(0.1e-6, 2e-6, 10)

res = open("resFIX_R.csv","w")
res.write("R,D,Analytic,Numerical\n")

for r2 in tqdm(r2s):
    writeRadius(r1, r2)
    runGMSH()
    runTTH()
    i = getCurrent()
    msfn_numerical = runNoiseEx("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk",i)
    msfn_analytical = Analytic(r1,r2*2)
    # write result to file
    res.write(f"{r1},{2*r2},{msfn_analytical},{msfn_numerical}")



res.close()

